In [1]:
import vault_client
import getpass
import tqdm

from os.path import expanduser
import json

def get_config():
    home = expanduser("~")
    
    with open(home + '/config.json') as f:
        config_data = f.read()
    config = json.loads(config_data)
    return config

yav = vault_client.instances.Production(
    rsa_login='miptgirl', 
    authorization='OAuth {}'.format(get_config()['vault_api_token']),)

ver = 'ver-01dssnc1zdezgbfd48xhj9j9xn'
YT_TOKEN = yav.get_version(ver)['value']['yt_token']
YQL_TOKEN = yav.get_version(ver)['value']['yql_token']
SOLOMON_TOKEN = yav.get_version(ver)['value']['solomon_token']
ver = 'ver-01eex0jhkrhzt14dfn8a2tyy42'
STATLOG_PASS = yav.get_version(ver)['value']['password']

def df_to_wiki(df, show_index = False):
    print('#|\n ||' + df.to_csv(index = show_index, sep = '|').replace('|', ' | ').replace('\n', '|| \n ||')[:-2] + '|#')

In [2]:
import io
import time
import pandas as pd
import datetime

HOST = 'http://mtgray03k.yandex.ru:8123'
HOST = 'http://mtlog01-01-1.yandex.ru:8123'
HOST = 'http://clickhouse.metrika.yandex.net:8123'
#HOST = 'http://mtsmart001-1.yandex.ru:8123'
HOST = 'http://mtch01k.metrika.yandex.net:8123'

import requests
import http.client  # or http.client if you're on Python 3
http.client._MAXHEADERS = 100000

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500, user = 'miptgirl', password = '1QwBgINk'):
    NUMBER_OF_TRIES = 30
    DELAY = 10
    
    for i in range(NUMBER_OF_TRIES):
        r = requests.post(host, 
                          params = {'timeout_before_checking_execution_speed': 120, 'max_bytes_in_join': 0}, 
                          timeout = connection_timeout, data = query,
                          auth = (user, password), verify = False
                         )
        if (r.status_code == 200) and ('DB:Exception' not in r.text):
            return r.text
        else:
            print('ATTENTION: try #%d failed' % i)
            if i != (NUMBER_OF_TRIES-1):
                #print query
                print(r.text)
                time.sleep(DELAY*(i+1))
            else:
                raise ValueError(r.text)
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500, user = 'miptgirl', password = '1QwBgINk'):
    data = get_clickhouse_data(query, host, connection_timeout, user, password) 
    df = pd.read_csv(io.StringIO(data), sep = '\t')
    return df

def upload(table, content, host=HOST):
    '''Uploads data to table in ClickHous'''
    content = content.encode('utf-8')
    query_dict = {
             'query': 'INSERT INTO ' + table + ' FORMAT TabSeparatedWithNames '
        }
    r = requests.post(host, data=content, params=query_dict, auth = ('miptgirl', '1QwBgINk'))
    result = r.text
    if r.status_code == 200:
        return result
    else:
        raise ValueError(r.text)

In [30]:
def get_data(date_str, host, counter_id, goal_id):

    q = '''
    select 
        CounterID as counter_id,
        arrayJoin(Goals.ID) as goal_id,
        UserID as user_id,
        sum(Sign) as conversions
    from visits_all
    where StartDate = '{date}'
        and UTCStartTime >= '2021-10-15 13:00:00'
        and UTCStartTime <= '2021-10-16 22:00:00'
        and CounterID = {counter_id}
        and goal_id = {goal_id}
    group by goal_id, counter_id, user_id
    format TSVWithNames
    '''.format(date = date_str, counter_id = counter_id, goal_id = goal_id)
    
    df = get_clickhouse_df(q, host)
    return df

In [31]:
counter_id = 92290
goal_id = 2890006

In [32]:
%%time 
prod_df = get_data('2021-10-16', 'http://mtgiga001-1.metrika.yandex.net:8123', counter_id, goal_id)

CPU times: user 4.32 ms, sys: 4.19 ms, total: 8.51 ms
Wall time: 177 ms


In [33]:
%%time 
test_df = get_data('2021-10-16', 'http://mtgiga001-1t.metrika.yandex.net:8123', counter_id, goal_id)

CPU times: user 7.64 ms, sys: 0 ns, total: 7.64 ms
Wall time: 85.9 ms


In [34]:
prod_df.merge(test_df, on = ['counter_id', 'goal_id', 'user_id'], suffixes = ('_prod', '_test'), 
             how = 'outer')

,counter_id,goal_id,user_id,conversions_prod,conversions_test
0,92290,2890006,9561066691547371520,4.0,NaN
1,92290,2890006,1164927931557345280,7.0,NaN
2,92290,2890006,6155726251620491264,1.0,NaN
3,92290,2890006,905417541618868480,2.0,NaN
4,92290,2890006,5619203901634406400,18.0,NaN
...,...,...,...,...,...
979,92290,2890006,1904549991630396160,2.0,NaN
980,92290,2890006,789263431613676928,2.0,NaN
981,92290,2890006,1168073221633457408,5.0,NaN
982,92290,2890006,7394425351631378432,3.0,NaN


In [36]:
q = '''
select 
    URL,
    GoalsReached,
    UTCEventTime
from merge.hits_v2
where EventDate >= '{date}'
    and CounterID = 92290
    and UserID = 5619203901634406400
format TSVWithNames
'''.format(date = '2021-10-15', counter_id = counter_id, goal_id = goal_id)

raw_prod_df = get_clickhouse_df(q, 'http://mtgiga001-1.metrika.yandex.net:8123')
raw_test_df = get_clickhouse_df(q, 'http://mtgiga001-1t.metrika.yandex.net:8123')

In [37]:
raw_prod_df

,URL,GoalsReached,UTCEventTime


In [38]:
raw_test_df

,URL,GoalsReached,UTCEventTime
